In [1]:
# !unzip robohearts.zip
%cd robohearts/

/content/robohearts


In [1]:
import gym
from gymhearts.Hearts import *
from gymhearts.Agent.human import Human
from gymhearts.Agent.random_agent import RandomAgent
from gymhearts.Agent.monte_carlo import MonteCarlo

In [5]:
NUM_ITERS = 10
NUM_EPISODES = 1000
TRAINING_ITERS = 10000
MAX_SCORE = 100
playersNameList = ['MonteCarlo', 'Rando', 'Randy', 'Randall']
agent_list = [0, 0, 0, 0]

# Human vs Random
agent_list[0] = MonteCarlo(playersNameList[0], {'print_info' : False})
agent_list[1] = RandomAgent(playersNameList[1], {'print_info' : False})
agent_list[2] = RandomAgent(playersNameList[2], {'print_info' : False})
agent_list[3] = RandomAgent(playersNameList[3], {'print_info' : False})

In [6]:
# who cares about output in training
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
weights = []
for trn_episode in range(TRAINING_ITERS):
    observation = env.reset()
    history = []
    while True:
        #env.render()

        now_event = observation['event_name']
        IsBroadcast = observation['broadcast']
        action = None
        if IsBroadcast == True:
            for agent in agent_list:
                agent.Do_Action(observation)
        else:
            playName = observation['data']['playerName']
            for agent in agent_list:
                if agent.name == playName:
                    action = agent.Do_Action(observation)

        # update my agent
        if now_event == 'RoundEnd':
            agent_list[0].update_weights(history, -reward['MonteCarlo'])
            history = []
            weights = agent_list[0].weight_vec
        if now_event == 'GameOver':
              break
        if not IsBroadcast and observation['data']['playerName'] == 'MonteCarlo':
            history.append(observation)
        observation, reward, done, info = env.step(action)


ROUND IS OVER
{'MonteCarlo': 6, 'Rando': 6, 'Randy': 14, 'Randall': 0}
[{'event_name': 'PassCards', 'broadcast': False, 'data': {'playerName': 'MonteCarlo', 'hand': ['7c', '8c', 'Tc', 'Jc', 'Kc', '3d', '7d', 'Kd', '5s', '8s', '9s', 'Ks', '5h']}}, {'event_name': 'ShowPlayerHand', 'broadcast': False, 'data': {'playerName': 'MonteCarlo', 'hand': ['Tc', 'Jc', 'Kc', 'Ac', '3d', '7d', 'Kd', 'Ad', '8s', '9s', 'Ks', '2h', '5h']}}, {'event_name': 'PlayTrick', 'broadcast': False, 'data': {'playerName': 'MonteCarlo', 'hand': ['Tc', 'Jc', 'Kc', 'Ac', '3d', '7d', 'Kd', 'Ad', '8s', '9s', 'Ks', '2h', '5h'], 'trickNum': 1, 'trickSuit': 'c', 'currentTrick': [{'playerName': 'Randy', 'card': '2c'}, {'playerName': 'Randall', 'card': '5c'}], 'IsHeartsBroken': False}}, {'event_name': 'PlayTrick', 'broadcast': False, 'data': {'playerName': 'MonteCarlo', 'hand': ['Jc', 'Kc', 'Ac', '3d', '7d', 'Kd', 'Ad', '8s', '9s', 'Ks', '2h', '5h'], 'trickNum': 2, 'trickSuit': 'Unset', 'currentTrick': [], 'IsHeartsBroken': 

AttributeError: 'MonteCarlo' object has no attribute 'get_value_fn'

In [5]:
# Evaluate the MC! Comment this line in for pretrained
# weights = [ [-0.46031637, -1.02296217, -1.64597146, 0.50871499, 0.05907032, -0.04527117,
# -2.67590788, -1.78400492, -0.08667306, 0.48108891, 0.66066313, -1.57675411,
# -0.56494518, -0.07736412, -0.3257198, -0.65003209, -0.63740714, 0.44494984,
# -0.1545964, 0.67457139, 2.31472314, 0.8694452, -2.29173301, 0.52783125,
# -0.86950875, -1.77655688, -3.29970913, -0.242993, -1.57548922, -1.34238258,
# 0.36816378, -3.23065985, -0.07919411, -2.1089143, -3.12815169, -0.74580836,
# 0.98398675, -0.75271283, -0.81051661, -0.60567687, -3.42010519, -0.63186969,
# -2.02352157, -0.27534069, -0.28736574, -1.15836776, -3.28679005, -0.33767846,
# -0.41568405, 0.2782292, -1.23761129, -1.80559854]
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
agent_list[0] = MonteCarlo(playersNameList[0], params={'weight_vec' : weights})
mc_wins = []
for i in range(NUM_ITERS):
  num_won = 0
  for i_episode in range(NUM_EPISODES):
      observation = env.reset()
      while True:
          now_event = observation['event_name']
          IsBroadcast = observation['broadcast']
          action = None
          if IsBroadcast == True:
              for agent in agent_list:
                  agent.Do_Action(observation)
          else:
              playName = observation['data']['playerName']
              for agent in agent_list:
                  if agent.name == playName:
                      action = agent.Do_Action(observation)
          if now_event == 'GameOver':
              num_won += int(observation['data']['Winner'] == 'MonteCarlo')
              break
          observation, reward, done, info = env.step(action)

  print("Finished iter: " + str(i))
  mc_wins.append(num_won)

Finished iter: 0
Finished iter: 1
Finished iter: 2
Finished iter: 3
Finished iter: 4
Finished iter: 5
Finished iter: 6
Finished iter: 7
Finished iter: 8
Finished iter: 9


In [6]:
# Evaluate the Random!
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
agent_list[0] = RandomAgent(playersNameList[0])
rand_wins = []
for i in range(NUM_ITERS):
  num_won = 0
  for i_episode in range(NUM_EPISODES):
      observation = env.reset()
      while True:
          now_event = observation['event_name']
          IsBroadcast = observation['broadcast']
          action = None
          if IsBroadcast == True:
              for agent in agent_list:
                  agent.Do_Action(observation)
          else:
              playName = observation['data']['playerName']
              for agent in agent_list:
                  if agent.name == playName:
                      action = agent.Do_Action(observation)
          if now_event == 'GameOver':
              num_won += int(observation['data']['Winner'] == 'MonteCarlo')
              break
          observation, reward, done, info = env.step(action)
  print("Finished iter: " + str(i))
  rand_wins.append(num_won)

Finished iter: 0
Finished iter: 1
Finished iter: 2
Finished iter: 3
Finished iter: 4
Finished iter: 5
Finished iter: 6
Finished iter: 7
Finished iter: 8
Finished iter: 9


In [7]:
print("The weights are: " + str(weights))
print("MonteCarlo won: " + str(mc_wins) + " times")
print("Random won: " + str(rand_wins) + " times")

The weights are: [-0.46031637 -1.02296217 -1.64597146  0.50871499  0.05907032 -0.04527117
 -2.67590788 -1.78400492 -0.08667306  0.48108891  0.66066313 -1.57675411
 -0.56494518 -0.07736412 -0.3257198  -0.65003209 -0.63740714  0.44494984
 -0.1545964   0.67457139  2.31472314  0.8694452  -2.29173301  0.52783125
 -0.86950875 -1.77655688 -3.29970913 -0.242993   -1.57548922 -1.34238258
  0.36816378 -3.23065985 -0.07919411 -2.1089143  -3.12815169 -0.74580836
  0.98398675 -0.75271283 -0.81051661 -0.60567687 -3.42010519 -0.63186969
 -2.02352157 -0.27534069 -0.28736574 -1.15836776 -3.28679005 -0.33767846
 -0.41568405  0.2782292  -1.23761129 -1.80559854]
MC won: [292, 307, 283, 286, 309, 311, 309, 296, 282, 295] times
Rand won: [248, 245, 242, 266, 255, 268, 273, 252, 251, 263] times
